<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/Validate_FRCNN_COCO2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# careful with dumb Python2 pycocotools:
# NameError: name 'unicode' is not defined in
# /usr/local/lib/python3.6/dist-packages/pycocotools/coco.py:308
# needs to be:
# if type(resFile) == str or (PYTHON_VERSION == 2 and type(resFile) == unicode):
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-tst5a44p
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-tst5a44p
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275279 sha256=ddaa9da609f879390dd42aca3c6c01175973db11251779c7e2eb1b989e0b43b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-v2rndxpo/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


In [2]:
%%time
# deps installation
try:
  import detectron2
except ImportError:
  !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
  !pip install -e detectron2_repo
  print('Stopping RUNTIME! Please run again.')
  import os
  os.kill(os.getpid(), 9)

CPU times: user 222 ms, sys: 53.7 ms, total: 276 ms
Wall time: 1.43 s


Create example model

In [0]:
import detectron2.model_zoo
from detectron2.config import get_cfg

In [0]:
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"

Create evaluator

In [0]:
# from detectron2.data import MetadataCatalog
# MetadataCatalog().get('coco_2014_val')
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [6]:
%%time
# No such file or directory: 'datasets/coco/annotations/instances_val2014.json'
import os
if not os.path.exists('datasets/coco/annotations/'):
  !mkdir -p datasets/coco
  !wget -q http://images.cocodataset.org/annotations/annotations_trainval2014.zip
  !unzip -d datasets/coco annotations_trainval2014.zip

CPU times: user 29 µs, sys: 7 µs, total: 36 µs
Wall time: 39.1 µs


In [0]:
evaluator = COCOEvaluator('coco_2014_val', cfg, False)

Run inference / metrics

In [0]:
from detectron2.data import build_detection_test_loader
# needed?
# cfg.DATASETS.TEST = ('coco_2014_val',)
data_loader = build_detection_test_loader(cfg, 'coco_2014_val')

In [0]:
from detectron2.engine import DefaultPredictor
predictor = DefaultPredictor(cfg)
# predictor is not a model:
# results = inference_on_dataset(predictor, data_loader, evaluator)

In [10]:
%%time
import os
if not os.path.exists('datasets/coco/val2014/'):
  !wget http://images.cocodataset.org/zips/val2014.zip
  !unzip val2014 -d datasets/coco/

CPU times: user 366 µs, sys: 50 µs, total: 416 µs
Wall time: 464 µs


In [11]:
%%time
results = inference_on_dataset(predictor.model, data_loader, evaluator)
results

Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=141.23s).
Accumulating evaluation results...
DONE (t=15.34s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.346
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.518
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.527
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets

In [12]:
!ls

annotations_trainval2014.zip  detectron2_repo  val2014.zip
datasets		      sample_data


Appendix: pycocotools Py3 error check

In [0]:
import pycocotools

In [14]:
pycocotools.__file__

'/usr/local/lib/python3.6/dist-packages/pycocotools/__init__.py'

In [17]:
!cat `dirname {pycocotools.__file__}`/coco.py | grep unicode

        if type(resFile) == str or (PYTHON_VERSION == 2 and type(resFile) == unicode):


In [19]:
results.keys()

odict_keys(['bbox', 'segm'])

In [20]:
results['bbox']

{'AP': 45.8080216334962,
 'AP-airplane': 59.591768325498094,
 'AP-apple': 25.307838084880395,
 'AP-backpack': 21.39432195257931,
 'AP-banana': 30.60160175400067,
 'AP-baseball bat': 37.49395523099272,
 'AP-baseball glove': 43.44074913308509,
 'AP-bear': 76.03213561543704,
 'AP-bed': 49.53013675811101,
 'AP-bench': 27.94417566068692,
 'AP-bicycle': 35.88971157372225,
 'AP-bird': 43.01590246236457,
 'AP-boat': 31.89311451915974,
 'AP-book': 15.523950564841165,
 'AP-bottle': 40.868324825889175,
 'AP-bowl': 43.761479990046105,
 'AP-broccoli': 33.34351660451721,
 'AP-bus': 63.96079665519164,
 'AP-cake': 42.76828766842542,
 'AP-car': 45.80246428394561,
 'AP-carrot': 28.32102069583968,
 'AP-cat': 71.04268172199484,
 'AP-cell phone': 40.64491928218635,
 'AP-chair': 32.62311147728073,
 'AP-clock': 54.12158674360007,
 'AP-couch': 41.41063652558003,
 'AP-cow': 58.555517331667374,
 'AP-cup': 45.770666827388794,
 'AP-dining table': 31.47628251280297,
 'AP-dog': 65.9292737208276,
 'AP-donut': 56.601

In [0]:
result_file = 'coco_val2014_rcnn_3x.json'
import json
with open(result_file, 'w') as f:
  f.write(json.dumps(results['bbox']))

In [0]:
from google.colab import files

files.download(result_file)